# Use Credit Risk Analytics Template

## Import Snowpark Pandas for dumping csv to Snowflake table

In [ ]:
#!pip install "snowflake-connector-python[pandas]"

In [1]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

# Code to read Credit Risk Analytics local files and dump the same into Snowflake

In [2]:
df_app_train = pd.read_csv("/data/application_train.csv")

In [3]:
df_app_train.shape

(307511, 122)

In [ ]:
df_install = pd.read_csv("/data/installments_payments.csv")

In [ ]:
df_cc_balance = pd.read_csv("/data/credit_card_balance.csv")

In [2]:
df_prev_app = pd.read_csv("/data/previous_application.csv")

In [2]:
df_bureau_bal = pd.read_csv("/data/bureau_balance.csv")

In [ ]:
df_bureau = pd.read_csv("/data/bureau.csv")

In [2]:
df_POS_CASH = pd.read_csv("/data/POS_CASH_balance.csv")

In [3]:
print ("bureau_balance ", df_bureau_bal.shape)

bureau_balance  (27299925, 3)


# Split the Bureau Balance due to huge volume throwing DEAD KERNEL issue while uploading on snowflake

In [4]:
import numpy as np
df_bureau_bal1, df_bureau_bal2,df_bureau_bal3 = np.array_split(df_bureau_bal, 3)

In [5]:
print ("bureau_balance_1 ", df_bureau_bal1.shape)
print ("bureau_balance_2 ", df_bureau_bal2.shape)
print ("bureau_balance_3 ", df_bureau_bal2.shape)

bureau_balance_1  (9099975, 3)
bureau_balance_2  (9099975, 3)
bureau_balance_3  (9099975, 3)


In [6]:
df_bureau_bal1.to_csv('/data/df_bureau_bal1.csv', index=False)
df_bureau_bal2.to_csv('/data/df_bureau_bal2.csv', index=False)
df_bureau_bal3.to_csv('/data/df_bureau_bal3.csv', index=False)

In [ ]:
print ("installments_payments ", df_install.shape)
print ("credit_card_balance ", df_cc_balance.shape)
print ("previous_application ", df_prev_app.shape)
print ("bureau_balance ", df_bureau_bal.shape)
print ("bureau ", df_bureau.shape)
print ("POS_CASH_balance ", df_POS_CASH.shape)

# Code to establish connection and  dump csv as a snowflake table in FDC setup

In [4]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_DATA_MANISH'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'PUBLIC'

In [5]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [6]:
session1.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session1.sql('use database FDC_DATA_MANISH;').collect()
session1.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

[Row(status='Statement executed successfully.')]

# Upload Application train data on snowflake

In [12]:
df_app_train['FONDKAPREMONT_MODE'].fillna("Nan values", inplace=True)
df_app_train['HOUSETYPE_MODE'].fillna("Nan values", inplace=True)

In [13]:
sf_df_app_train=session1.createDataFrame(
        df_app_train.values.tolist(),
        schema=df_app_train.columns.tolist())
sf_df_app_train.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_TRAIN")

TypeError: field WALLSMATERIAL_MODE: Cannot merge type <class 'snowflake.snowpark.types.StringType'> and <class 'snowflake.snowpark.types.FloatType'>

# Upload Installments Payments data on snowflake

In [ ]:
sf_df_install=session1.createDataFrame(
        df_install.values.tolist(),
        schema=df_install.columns.tolist())
sf_df_install.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_installments_payments")

# Upload Credit Card Balance data on snowflake

In [ ]:
sf_df_cc_balance=session1.createDataFrame(
        df_cc_balance.values.tolist(),
        schema=df_cc_balance.columns.tolist())
sf_df_cc_balance.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_credit_card_balance")

# Handle NaN values and Upload Previous Application data on snowflake

In [6]:
df_prev_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [10]:
df_prev_app['NAME_TYPE_SUITE'].value_counts()

NAME_TYPE_SUITE
nan                820405
Unaccompanied      508970
Family             213263
Spouse, partner     67069
Children            31566
Other_B             17624
Other_A              9077
Group of people      2240
Name: count, dtype: int64

In [17]:
df_prev_app['NAME_TYPE_SUITE'].fillna("Nan values", inplace=True)

In [22]:
df_prev_app['NAME_TYPE_SUITE'].value_counts()

NAME_TYPE_SUITE
Nan Values         820405
Unaccompanied      508970
Family             213263
Spouse, partner     67069
Children            31566
Other_B             17624
Other_A              9077
Group of people      2240
Name: count, dtype: int64

In [26]:
df_prev_app['PRODUCT_COMBINATION'].fillna("Nan Values", inplace=True)

In [27]:
df_prev_app['PRODUCT_COMBINATION'].value_counts()

PRODUCT_COMBINATION
Cash                              285990
POS household with interest       263622
POS mobile with interest          220670
Cash X-Sell: middle               143883
Cash X-Sell: low                  130248
Card Street                       112582
POS industry with interest         98833
POS household without interest     82908
Card X-Sell                        80582
Cash Street: high                  59639
Cash X-Sell: high                  59301
Cash Street: middle                34658
Cash Street: low                   33834
POS mobile without interest        24082
POS other with interest            23879
POS industry without interest      12602
POS others without interest         2555
Nan Values                           346
Name: count, dtype: int64

In [28]:
sf_df_prev_app=session1.createDataFrame(
        df_prev_app.values.tolist(),
        schema=df_prev_app.columns.tolist())
sf_df_prev_app.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_previous_application")

# Upload Bureau Balance data files on snowflake

In [10]:
sf_df_bureau_bal1=session1.createDataFrame(
        df_bureau_bal1.values.tolist(),
        schema=df_bureau_bal1.columns.tolist())
sf_df_bureau_bal1.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_bureau_balance")

In [5]:
#df_bureau_bal2 = pd.read_csv("/data/df_bureau_bal2.csv")
sf_df_bureau_bal2=session1.createDataFrame(
        df_bureau_bal2.values.tolist(),
        schema=df_bureau_bal2.columns.tolist())
sf_df_bureau_bal2.write.mode("append").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_bureau_balance")

In [5]:
#df_bureau_bal3 = pd.read_csv("/data/df_bureau_bal3.csv")
sf_df_bureau_bal3=session1.createDataFrame(
        df_bureau_bal3.values.tolist(),
        schema=df_bureau_bal3.columns.tolist())
sf_df_bureau_bal3.write.mode("append").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_bureau_balance")

# Upload Bureau data on snowflake

In [ ]:
sf_df_bureau=session1.createDataFrame(
        df_bureau.values.tolist(),
        schema=df_bureau.columns.tolist())
sf_df_bureau.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_bureau")

# Upload POS CASH Balance data on snowflake

In [6]:
sf_df_POS_CASH=session1.createDataFrame(
        df_POS_CASH.values.tolist(),
        schema=df_POS_CASH.columns.tolist())
sf_df_POS_CASH.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_POS_CASH_balance")